In [4]:
%pip install "git+https://github.com/flyteorg/flytekit@3e3ec17ea" union "pydantic>2" pandas pyarrow

  Cloning https://github.com/flyteorg/flytekit (to revision 3e3ec17ea) to /private/var/folders/4q/frdnh9l10h53gggw1m59gr9m0000gp/T/pip-req-build-cdnu3ehs
  Running command git clone --filter=blob:none --quiet https://github.com/flyteorg/flytekit /private/var/folders/4q/frdnh9l10h53gggw1m59gr9m0000gp/T/pip-req-build-cdnu3ehs
  Running command git checkout -q 3e3ec17ea
  Resolved https://github.com/flyteorg/flytekit to commit 3e3ec17ea
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [8]:
import flytekit as fk
import pandas as pd
import sys


image = fk.ImageSpec(
    name="jupyter-notebook-workshop",
    packages=[
        "pandas",
        "pyarrow",
        "git+https://github.com/flyteorg/flytekit@3e3ec17ea",
        "union==0.1.92",
    ],
    apt_packages=["git"],
    python_version=f"{sys.version_info.major}.{sys.version_info.minor}",
)

task = fk.task(container_image=image)


@task
def get_df() -> pd.DataFrame:
    """Returns a new Dataframe with cols [Name, Age, Grade and PassedTest]"""
    return pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
    'Age': [23, 25, 22, 24, 23],
    'Grade': ['A', 'B', 'A', 'C', 'B'],
    'PassedTest': [True, False, True, False, True]
})

In [9]:
from union.remote import UnionRemote
from union.ucimage._image_builder import _register_union_image_builder

_register_union_image_builder() 
remote = UnionRemote()

16:45:33.318861 WARNING  remote.py:286 - Jupyter notebook and interactive task  
                         support is still alpha.

In [10]:
exe = remote.execute(get_df, inputs={})
exe

Image cr.union.ai/jupyter-notebook-workshop:Y6BvxH4QmsZJtXDChYQf7w not found.
🐳 Build not found, submitting a new build...
👍 Build submitted!
⏳ Waiting for build to finish at: https://serverless.union.ai/org/cosmicbboy/projects/system/domains/development/executions/a4pxnr8tfng4hzm7b9dc
✅ Build completed in 0:00:25!


Flyte Serialized object (FlyteWorkflowExecution):
  id:
    project: flytesnacks
    domain: development
    name: arrsff5vfq26t7wgz6zk
  spec:
    launch_plan:
      resource_type: 1
      project: flytesnacks
      domain: development
      name: get_df
      version: RL8n60U4uGhZInqtav_jfw
    metadata:
      principal: 00ug8tl4tit8Dvpi65d7
  closure:
    created_at:
      seconds: 1730493966
      nanos: 464276000
    updated_at:
      seconds: 1730493966
      nanos: 464276000

In [12]:
exe.wait(poll_interval=1)

Flyte Serialized object (FlyteWorkflowExecution):
  id:
    project: flytesnacks
    domain: development
    name: arrsff5vfq26t7wgz6zk
  spec:
    launch_plan:
      resource_type: 1
      project: flytesnacks
      domain: development
      name: get_df
      version: RL8n60U4uGhZInqtav_jfw
    metadata:
      principal: 00ug8tl4tit8Dvpi65d7
  closure:
    outputs:
      uri: s3://unionai-production-serverless-us-east- [...]
    phase: 4
    started_at:
      seconds: 1730493967
      nanos: 88030000
    duration:
      seconds: 24
      nanos: 343946000
    created_at:
      seconds: 1730493966
      nanos: 464276000
    updated_at:
      seconds: 1730493991
      nanos: 431976000

In [13]:
dataframe = exe.outputs['o0']

In [14]:
dataframe

,Name,Age,Grade,PassedTest
0,Alice,23,A,True
1,Bob,25,B,False
2,Charlie,22,A,True
3,David,24,C,False
4,Eva,23,B,True


In [15]:
def local_function(dataframe: pd.DataFrame):
    return dataframe.groupby("Grade")["Age"].sum()

local_function(dataframe)

Grade
A    45
B    48
C    24
Name: Age, dtype: int64

In [16]:
from flytekit.types.file import FlyteFile


training_image = fk.ImageSpec(
    name="jupyter-notebook-workshop-training",
    packages=["pandas", "pyarrow", "joblib", "scikit-learn", "git+https://github.com/flyteorg/flytekit@3e3ec17ea"],
    apt_packages=["git"],
    python_version=f"{sys.version_info.major}.{sys.version_info.minor}",
)

training_task = fk.task(container_image=training_image)

@training_task
def train_model(dataframe: pd.DataFrame) -> FlyteFile:
    import joblib
    from sklearn.linear_model import LogisticRegression

    model = LogisticRegression()
    model.fit(dataframe[["Age"]], dataframe["PassedTest"])

    with open("model.pkl", "wb") as f:
        joblib.dump(model, f)

    return FlyteFile(path="model.pkl")


@fk.workflow
def train_wf(dataframe: pd.DataFrame) -> FlyteFile:
    return train_model(dataframe)

In [17]:
model_exe = remote.execute(train_wf, inputs={"dataframe": dataframe})
model_exe

Image cr.union.ai/jupyter-notebook-workshop-training:NZgUU3lWlNGjEPbANb6CUw found. Skip building.


Flyte Serialized object (FlyteWorkflowExecution):
  id:
    project: flytesnacks
    domain: development
    name: a5vll66rkl8l252w8rpz
  spec:
    launch_plan:
      resource_type: 3
      project: flytesnacks
      domain: development
      name: train_wf
      version: fjvzk4vXgh-x4JHvRwsc3w
    metadata:
      principal: 00ug8tl4tit8Dvpi65d7
  closure:
    created_at:
      seconds: 1730494255
      nanos: 183052000
    updated_at:
      seconds: 1730494255
      nanos: 183052000

In [18]:
model_exe.wait(poll_interval=1)
model = model_exe.outputs['o0']
model

╭─────────────────────────────────────── Traceback (most recent call last) ───────────────────────────────────────╮
│ in <module>:2                                                                                                   │
│                                                                                                                 │
│ ❱ 2 model = model_exe.outputs['o0']                                                                             │
│                                                                                                                 │
│ /Users/nielsbantilan/miniforge3/envs/jupyter-notebook-workshop/lib/python3.11/site-packages/flytekit/remote/exe │
│ cutions.py:154 in outputs                                                                                       │
│                                                                                                                 │
│ ❱ 154 │   │   outputs = super().outputs                                                                         │
│                                                                                                                 │
│ /Users/nielsbantilan/miniforge3/envs/jupyter-notebook-workshop/lib/python3.11/site-packages/flytekit/remote/exe │
│ cutions.py:47 in outputs                                                                                        │
│                                                                                                                 │
│ ❱  47 │   │   │   raise user_exceptions.FlyteAssertion(                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
FlyteAssertion: USER:AssertionError: error=Outputs could not be found because the execution ended in failure. Error
message: Trace:

    Traceback (most recent call last):
      File "/opt/micromamba/envs/runtime/lib/python3.11/site-packages/flytekit/core/base_task.py", line 741, in 
dispatch_execute
        native_inputs = self._literal_map_to_python_input(input_literal_map, exec_ctx)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
      File "/opt/micromamba/envs/runtime/lib/python3.11/site-packages/flytekit/core/base_task.py", line 610, in 
_literal_map_to_python_input
        return TypeEngine.literal_map_to_kwargs(ctx, literal_map, self.python_interface.inputs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
      File "/opt/micromamba/envs/runtime/lib/python3.11/site-packages/flytekit/core/utils.py", line 308, in wrapper
        return func(*args, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^
      File "/opt/micromamba/envs/runtime/lib/python3.11/site-packages/flytekit/core/type_engine.py", line 1420, in 
literal_map_to_kwargs
        return synced(ctx, lm, python_types, literal_types)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
      File "/opt/micromamba/envs/runtime/lib/python3.11/site-packages/flytekit/utils/asyn.py", line 100, in wrapped
        return self.run_sync(coro_func, *args, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
      File "/opt/micromamba/envs/runtime/lib/python3.11/site-packages/flytekit/utils/asyn.py", line 93, in run_sync
        return self._runner_map[name].run(coro)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
      File "/opt/micromamba/envs/runtime/lib/python3.11/site-packages/flytekit/utils/asyn.py", line 72, in run
        res = fut.result(None)
              ^^^^^^^^^^^^^^^^
      File "/opt/micromamba/envs/runtime/lib/python3.11/concurrent/futures/_base.py", line 456, in result
        return self.__get_result()
               ^^^^^^^^^^^^^^^^^^^
      File "/opt/micromamba/envs/runtime/lib/python3.11/concurrent/futures/_base.py", line 401, in __get_result
        raise self._exception
      File "/opt/micromamba/envs/runtime/lib/python3.11/site-packages